# Projeto de Machine Learning

## 1. Case RH - Retenção de funcionários
Objetivo:
Criar um modelo que represente o *Job Satisfaction* de um profissional de TI a partir de características da compania e outros dados obtidos pela pesquisa. Utilize os dados da pesquisa de 2018 do Stackoverflow.

## 2. Case Empresa de Anúncios - Previsão de salários

Objetivo:
Criar um modelo que represente o *Salary* de um profissional de TI a partir de características da compania e outras características dos funcionários utilizando os dados disponibilizados na pesquisa de 2017 do Stackoverflow.


### Passos:

0. Carregar base de dados
1.Seleção de features - Análise das Features / Construir base analítica
  - remover linhas com missing
  - codificar variáveis categóricas como fatores
  - etc.
2. Análise exploratória da Base:
  - Histograma de Salários
  - Histograma de satisfação - Quantos tem satisfação maior que 0.7 
  - Correlações das features, 
  - etc.
4. Traduzir o problema - buscar a melhor solução de negócio
5. Selecionar e treinar o modelo 
  - Selecionar o modelo
  - Definir X (features) e y (variável dependente)
  - Normalizar as features (facultativo, mas melhora os resultados de predição)
  - Separar modelos em treino e teste 
  -Treinar o modelo
6. Retornar MSE para o modelo e distribuição real do seu y_teste e do seu y_pred (y preditos)



# LOAD DATASET

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
from sklearn.model_selection import train_test_split

from sklearn import metrics
pd.set_option('display.max_rows', 3500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

data = pd.read_csv("../survey_results_public.csv")

# TRATAMENTO DA BASE

#### Elimino linhas que tenham nulos nas colunas Salário, JobSatisfaction, JobSeekingStatus e CarreerSatisfaction

In [ ]:
data2 = data[~(data.Salary.isnull() | 
        data.JobSatisfaction.isnull() |
        data.CareerSatisfaction.isnull())]
nrow = data2.shape[0]
print("# Linhas :", nrow)
print("# Colunas: ", data2.shape[1])
del data

#### Deleto linhas com HoursPerWeek >= 20 ( EmploymentStatus full-time ou part-time) 
(Supondo erro no preenchimento da survey)

In [ ]:
data2 = data2[(data2.HoursPerWeek<20)]
print(len(data2))

#### Deleto colunas que possuam mais de 5 mil missing (90%)

In [ ]:
del_columns = data2.columns[data2.isnull().sum() >5000]
df = data2.drop(del_columns, axis = 1)
del_columns

In [ ]:
#PRINT DO NUMERO DE MISSING POR COLUNA:

#print("data_tratada shape: ", df.shape)
#print("total missing data")
#print()
#for col in df.columns:
#    print(col , df[col].isnull().sum())

#### Deleto algumas colunas que são irrelevantes, que ainda possuem alto valor de missing ou que só possuem um só tipo de resposta (ex: Professional)

In [ ]:
#Colunas irrelevantes
df = df.drop("Respondent", axis = 1)
df = df.drop('PronounceGIF', axis = 1)
df = df.drop('ClickyKeys', axis = 1)

#Colunas com alto valor de Missing Data
df = df.drop('HaveWorkedDatabase', axis =1)
df = df.drop('WantWorkDatabase', axis =1)
df = df.drop('HaveWorkedPlatform', axis =1)
df = df.drop('WantWorkPlatform', axis =1)

#Coluna com apenas uma só resposta
df = df.drop("Professional", axis = 1)

#### Transformo as colunas com múltiplas respostas em dummies


HaveWorkedLanguage, WantWorkLanguage, IDE, DeveloperType e ImportantBenefits, Gender, Race, StackOverflowDevices, MetricAssess, EducationTypes


In [ ]:
def strip_lista(lista):
    return list(map(str.strip,lista))

def get_dummy_cols(df, column_string, prefix = "uses"):
    df1 = df[column_string].str.split(';').dropna()
    df1 = df1.apply(lambda x: strip_lista(x))
    df2 = pd.get_dummies(df1.apply(pd.Series).stack()).sum(level=0)
    df = df.drop(column_string, axis = 1)
    df2.columns = [prefix +"_"+ x for x in df2.columns]     
    result = pd.merge(df,df2,left_index=True,right_index=True)
    return result


df = get_dummy_cols(df,'HaveWorkedLanguage', prefix ='Worked_with')
df = get_dummy_cols(df,'WantWorkLanguage', prefix = 'Want_work')
df = get_dummy_cols(df,'IDE' ,prefix = 'Uses')
df = get_dummy_cols(df, "DeveloperType", prefix = "DevType")
df = get_dummy_cols(df,'ImportantBenefits' ,prefix = 'IsImportantBenefit')
df = get_dummy_cols(df,'Gender' ,prefix = 'Gender')
df = get_dummy_cols(df, 'Race', prefix = "Race")
df = get_dummy_cols(df, 'StackOverflowDevices', prefix = "Uses_StackOverflow_in")
df = get_dummy_cols(df, 'MetricAssess', prefix = "MetricAssess_")
df = get_dummy_cols(df, 'EducationTypes', prefix = "Education")

print(df.shape)
df.head(1)

### FIM DO PRE TRATAMENTO - Salvo a base pré tratada
